# Transportation problem example

|            |    Retail 1   |   Retail  2   |   Retail 3    |    Supply   |
| :-------:  | :----------:  | :----------:  | :---------:   | :---------: |
|  Supplier 1|       10      |      9        |       15      |    100      |
|  Supplier 2|       7       |      19       |       9       |    50      |
|  Supplier 3|       11      |      21       |       11      |    60      |
|  Supplier 4|       15      |      8        |       10      |    40      |
|  Demand    |      100      |     100       |       50      |            |

$$
\begin{aligned}
\min z & = 10x_{11} + 9x_{12} + 15x_{13} + \ldots + 15x_{41} + 8x_{42} + 10x_{43} \\
\text{Subjecto to:} \\
& \sum_{i=1}^{4} x_{i1} + x_{i2} + x_{i3} = \text{Supply}_i \\ 
& \sum_{j=1}^{3} x_{1j} + x_{2j} + x_{3j} + x_{4j} = \text{Demand}_j \\ 
& x_{ij} \ge 0 \\
& i = 1,2,3,4 \\
& j = 1,2,3
\end{aligned}    
$$

In [49]:
using JuMP

In [50]:
using GLPK

In [51]:
m = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [52]:
c = [10, 7, 11, 15, 9, 19, 21, 8, 15, 9, 11, 10 ]
supply = [100, 50, 60, 40]
demand = [100, 100, 50]

3-element Vector{Int64}:
 100
 100
  50

In [53]:
sum(supply) == sum(demand)

true

In [54]:
@variable(m, x[1:4, 1:3])

4×3 Matrix{VariableRef}:
 x[1,1]  x[1,2]  x[1,3]
 x[2,1]  x[2,2]  x[2,3]
 x[3,1]  x[3,2]  x[3,3]
 x[4,1]  x[4,2]  x[4,3]

In [55]:
@objective(m, Min, sum(c .* x[:]))

10 x[1,1] + 7 x[2,1] + 11 x[3,1] + 15 x[4,1] + 9 x[1,2] + 19 x[2,2] + 21 x[3,2] + 8 x[4,2] + 15 x[1,3] + 9 x[2,3] + 11 x[3,3] + 10 x[4,3]

In [56]:
for i in 1:4
    @constraint(m, sum(x[i,:]) == supply[i])
end

In [57]:
for j in 1:3
    @constraint(m, sum(x[:,j]) == demand[j])
end

In [58]:
@constraint(m, x[:] .>= 0);

In [59]:
println(m)

Min 10 x[1,1] + 7 x[2,1] + 11 x[3,1] + 15 x[4,1] + 9 x[1,2] + 19 x[2,2] + 21 x[3,2] + 8 x[4,2] + 15 x[1,3] + 9 x[2,3] + 11 x[3,3] + 10 x[4,3]
Subject to
 x[1,1] + x[1,2] + x[1,3] = 100.0
 x[2,1] + x[2,2] + x[2,3] = 50.0
 x[3,1] + x[3,2] + x[3,3] = 60.0
 x[4,1] + x[4,2] + x[4,3] = 40.0
 x[1,1] + x[2,1] + x[3,1] + x[4,1] = 100.0
 x[1,2] + x[2,2] + x[3,2] + x[4,2] = 100.0
 x[1,3] + x[2,3] + x[3,3] + x[4,3] = 50.0
 x[1,1] ≥ 0.0
 x[2,1] ≥ 0.0
 x[3,1] ≥ 0.0
 x[4,1] ≥ 0.0
 x[1,2] ≥ 0.0
 x[2,2] ≥ 0.0
 x[3,2] ≥ 0.0
 x[4,2] ≥ 0.0
 x[1,3] ≥ 0.0
 x[2,3] ≥ 0.0
 x[3,3] ≥ 0.0
 x[4,3] ≥ 0.0



In [60]:
optimize!(m)

In [61]:
value.(x)

4×3 Matrix{Float64}:
 40.0  60.0   0.0
 50.0   0.0   0.0
 10.0   0.0  50.0
  0.0  40.0   0.0

|            |    Retail 1   |   Retail  2   |   Retail 3    |    Supply   |
| :-------:  | :----------:  | :----------:  | :---------:   | :---------: |
|  Supplier 1|       10 (40) |   9 (60)      |   15          |    100      |
|  Supplier 2|       7  (50) |   19          |       9       |    50      |
|  Supplier 3|       11 (10) |   21          |   11 (50)     |    60      |
|  Supplier 4|   15          |   8 (40)      |       10      |    40      |
|  Demand    |      100      |     100       |       50      |            |

In [64]:
totalcost = 10 * 40 + 9 * 60 + 7 * 50 + 11 * 10 + 11 * 50 + 8 * 40 

2270

In [65]:
solution_summary(m)

* Solver : GLPK

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Message from the solver:
  "Solution is optimal"

* Candidate solution
  Objective value      : 2270.0
  Objective bound      : -Inf
  Dual objective value : 2270.0

* Work counters
  Solve time (sec)   : 0.00011
